In [ ]:
import math
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text

In [ ]:
# Setup: use the connection_url variable provided by the notebook environment
engine = create_engine(connection_url)

In [ ]:
# Step 1: Get Last Processed Timestamp
last_ts_query = "SELECT last_processed_ts FROM fraud_metadata WHERE id = 1;"
last_ts_row = pd.read_sql(last_ts_query, engine)
last_ts = last_ts_row.iloc[0]['last_processed_ts']

In [ ]:
# Step 2: Fetch New Transactions
query = f"""
SELECT * FROM transactions
WHERE ts > '{last_ts}'
ORDER BY user_id, ts;
"""
df = pd.read_sql(query, engine)

In [ ]:
# Haversine Distance Function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [ ]:
# Step 3: Fraud Detection Logic
frauds = []
if not df.empty:
    grouped = df.groupby('user_id')

    for user_id, txs in grouped:
        txs = txs.sort_values('ts')
        for i in range(1, len(txs)):
            t1 = txs.iloc[i - 1]
            t2 = txs.iloc[i]

            dist = haversine(t1['lat'], t1['lon'], t2['lat'], t2['lon'])
            secs = (t2['ts'] - t1['ts']).total_seconds()
            if secs <= 0:
                continue
            speed = dist / (secs / 3600.0)

            if speed > 500:
                frauds.append((
                    int(t1['tx_id']), int(t2['tx_id']), int(user_id),
                    dist, int(secs), speed
                ))

    # Step 4: Insert into potential_fraud
    if frauds:
        insert_df = pd.DataFrame(frauds, columns=[
            'tx_id1', 'tx_id2', 'user_id', 'distance_km', 'secs', 'speed_kmh'
        ])
        insert_df.to_sql('potential_fraud', con=engine, if_exists='append', index=False)

    # Step 5: Update Metadata Timestamp
    new_last_ts = df['ts'].max()
    update_query = f"""
    UPDATE fraud_metadata
    SET last_processed_ts = '{new_last_ts}'
    WHERE id = 1;
    """
    with engine.connect() as conn:
        conn.execute(text(update_query))
else:
    print("No new transactions to process.")